# Demo for bruk av openAI med cv data fra CV partner

Dette er ment for å være en rask demo for å vise hvordan openai apiet fungerer, hvordan man kan laste inn CV data, og gi alle et godt utgangspunkt for hvordan å jobbe med ChatGPT gjennom et API

In [23]:
import os
from openai import OpenAI
from dotenv import load_dotenv

Her bruker vi noe som heter dotenv for å laste miljø variabler fra en .env fil som ligger i root dir. Dette er en enkel måte å organisere miljøvariabler på per prosjekt, og man slipper all styret man typisk har i de forskjellige operativ systemene (Windows spesielt).

Denne biten med kode antar at det finnes en fil i root med navn `.env`. I den filen bør det være en linje `OPENAI_API_KEY = ...` der man fyller in API nøkkelen. Denne filen ligger i .gitignore så derre må lage den selv.

In [14]:
load_dotenv("../.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [26]:
Rolle_beskrivelse = """1   Navn på oppdrag /
prosjekt
LLM Engineer (mulig teamleder) til nytt KI Team
2   Kort beskrivelse av oppdraget
Sikt etablerer et nytt team som jobber med kunstig intelligens på tvers av virksomheten. Dette
vil støtte produkt-teamene i Sikt med kompetanse og gjennomføringskraft og realisere piloter
på ny og bedre funksjonalitet.
Vi ønsker med dette teamet og etablerere et sterkt fagmiljø for data science og KI i Sikt som
gjør oss i stand til å identifisere, planlegge og utnytte flere av mulighetene som KI åpner for i
produktene våre.
Selv om hovedoppgaven til teamet blir å realisere faktiske produkter og ny funksjonalitet, blir
det også en svært viktig å jobbe sammen med produkt-teamene for å identifisere de riktige
initiativene, samt å dokumentere og formidle både internt og eksternt om både læring, feil og
suksesser.
Vi har identifisert to ulike roller som blir viktig for oss fra starten, og hvor vi søker en erfaren
konsulent. En av disse rollene vil også ha funksjon som teamleder. Til stillingen som «LLM
engineer» («Large Language Model engineer») søker vi en erfaren konsulent med følgende
kvalifikasjoner:
Kvalifikasjoner
(dersom aktuell som teamleder) Erfaring med å lede og inspirere høytytende team
for å fremme et positivt arbeidsmiljø og høy leveransekraft.
\n
Evnen til å veilede og utvikle team-medlemmer, både faglig og personlig, for å styrke
teamets kompetanse og karriereutvikling.
\n
Kommunikasjon: Sterke muntlige og skriftlige kommunikasjonsevner for å forklare
komplekse analyser og konklusjoner til et ikke-teknisk publikum.
\n
Dyptgående programmeringskunnskap, fortrinnsvis Python.
\n
Praktisk erfaring og utforsking av innovativ bruk av LLM via API det siste året.
\n
Kunnskap og erfaring med open source språkmodeller, og fine-tuning av disse.
\n
Erfaring med RAG (Retrieval Augmented Generation) med LangChain eller
tilsvarende.
\n
Grunnleggende forståelse for maskinlæring og NLP (Natural Language Processing)
\n
Erfaring med å utvikle og vedlikeholde API-er.
\n
God forståelse og erfaring med git, CI/CD, skyteknologi og Docker eller kubernetes.
\n
Teamarbeid: Evnen til å jobbe effektivt i tverrfaglige team og samarbeide med andre
fagområder.
\n
Nysgjerrighet og lærevillighet: En evne til å holde seg oppdatert med de siste
trendene og teknologiene innen AI og maskinlæring.
\n
God skriftlig og muntlig fremstillingsevne på både norsk og engelsk
Arbeidsoppgaver:
\n
Sammen med produktteamene idémyldre, identifisere og planlegge nye piloter og
initiativer.
\n
Sammen med teamet utvikle ny funksjonalitet og tjenester med LLM og generativ AI.
\n
Formidling og historiefortelling om arbeidet, prosessen og resultatene underveis
\n
Bidra til godt samarbeid i produktteamet
Personlige egenskaper:
\n
Gode analytiske ferdigheter
\n
Høy gjennomføringsevne
\n
Presis i kommunikasjon
\n
Svært gode samarbeidsevner"""

In [27]:
client = OpenAI()
CV = client.files.create(
  file=open("../data/AskJuhlMarkestad_pdf_cv.pdf", "rb"),
  purpose='assistants'
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Du hjelper konsulenter med å spisse deres CV intro text for en gitt rolle beskrivelse."},
    {"role": "user", "content": f"Skriv en CV intro tekst på 150 ord for en gitt CV og en gitt rolle beskrivelse: \n CV: {CV.id} \n Rolle Beskrivelse: {Rolle_beskrivelse}"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Jeg er en erfaren LLM Engineer med dokumentert suksess i å lede og inspirere høytytende team for å oppnå høy leveransekraft og et positivt arbeidsmiljø. Med en dyp forståelse for maskinlæring og NLP, kombinert med dyptgående programmeringskunnskaper, spesielt i Python, har jeg utforsket innovativ bruk av LLM via API det siste året. Jeg har også solid erfaring med fine-tuning av open source språkmodeller og RAG (Retrieval Augmented Generation) teknologier som LangChain.\n\nMin evne til å veilede og utvikle teammedlemmer har bidratt til å styrke lagets kompetanse, og min sterke kommunikasjonsevne gjør at jeg effektivt kan formidle komplekse analyser til et ikke-teknisk publikum. Med en nysgjerrig og lærevillig tilnærming holder jeg meg oppdatert på de nyeste AI- og maskinlæringstrendene.\n\nJeg er presis i kommunikasjon, har gode analytiske ferdigheter og trives i tverrfaglige teammiljøer. Med en høy gjennomføringsevne og gode samarbeidsevner er jeg klar ti

In [41]:
client.files.list(purpose="assistants")

SyncPage[FileObject](data=[], object='list', has_more=False)

In [40]:
for file in client.files.list(purpose="assistants"):
    client.files.delete(file.id)